In [1]:
import pandas as pd
import os
from functools import reduce

#setup file import
schools_file = os.path.join("raw_data", "schools_complete.csv")
students_file = os.path.join("raw_data", "students_complete.csv")


# read files into dataframes
schools_df = pd.read_csv(schools_file)
students_df = pd.read_csv(students_file)

#Change the 'name' column of schools_df to 'school'.
# This will make merging and accessing the column more intuitive later,
# and keeps the dataset consistent throughout the entire analysis.
schools_df = schools_df.rename(columns={"name":"school"})

In [2]:
schools_df

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [3]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary


In [4]:
'''
District Summary
Create high-level snapshot (in table form) of district's key metrics, including:
- total schools
- total students (formatted with thousands separators)
- total budget (formatted as currency)
- average math score 
- average reading score
- % passing math
- % passing reading
- overall passing rate (average of % passing math & % passing reading)
'''

#calculate total number of schools by counting school ids
total_schools = schools_df['School ID'].count()

#calculate total number of students by counting student ids
total_students = students_df['Student ID'].count()

#calculate total budget for each school
total_budget = schools_df['budget'].sum()

#calculate average math score for all students district-wide
avg_math_score = students_df['math_score'].mean()

#calculate average reading score for all students district-wide
avg_reading_score = students_df['reading_score'].mean()

#calculate number of students with passing math scores -- create data frame with passing scores,
# count number of students in dataframe, then calculate by dividing value by total_students
passing_math = students_df.loc[students_df["math_score"] >= 70, ["math_score"]]
pct_passing_math = (passing_math['math_score'].count() / total_students) * 100

#calculate number of students with passing read scores -- create data frame with passing scores,
# count number of students in dataframe, then calculate by dividing value by total_students
passing_reading = students_df.loc[students_df["reading_score"] >= 70, ["reading_score"]]
pct_passing_reading = (passing_reading['reading_score'].count() / total_students) * 100

# Average of % passing reading and % passing match
overall_passing_rate = ((pct_passing_math + pct_passing_reading) / 2)
overall_passing_rate

# create district_summary dataframe for display
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [pct_passing_math],
    "% Passing Reading": [pct_passing_reading],
    "Overall Passing Rate": [overall_passing_rate]})
district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                                    "Average Reading Score", "% Passing Math", "% Passing Reading", 
                                    "Overall Passing Rate"]]
#work_df_us["avg pledge per project"] = work_df_us["avg pledge per project"].map("${:,.2f}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [28]:
# create dataframe containing only students who pass math
students_passing_math = students_df.loc[students_df['math_score'] > 70]

# reduce dataframe to include only high school name and a new field that will be used by the groupby for the 
# student counter. For now, it is initialized to zero.
students_passing_math_red = pd.DataFrame({"school": students_passing_math["school"],
                                         "nbr_students_passing_math": 0})

#place school field in leftmost position for easier visualization by hoo-mans
students_passing_math_red = students_passing_math_red[["school","nbr_students_passing_math"]]


# group students passing math by high school and count. The count of students in the students_passing_math_red 
# dataset will go in the nbr_students_passing_math column.
students_passing_math_by_school = students_passing_math_red.groupby(['school']).count().reset_index()

# create dataframe containing only students who pass reading
students_passing_reading = students_df.loc[students_df['reading_score'] > 70]

# reduce dataframe to include only high school and a new, empty field to contain the student counter created
# by the groupby in the next step.
students_passing_reading_red = pd.DataFrame({"school": students_passing_reading["school"],
                                          "nbr_students_passing_reading": 0})
# move school to leftmost position to make it easier for hoo-mans to visualize
students_passing_reading_red = students_passing_reading_red[["school","nbr_students_passing_reading"]]

# group students passing reading by high school and count
students_passing_reading_by_school = students_passing_reading_red.groupby(['school']).count().reset_index()

#group students by school to calculate average math and reading scores per school
school_group = students_df.groupby(['school'])

# build dataframe with calculated average score. reset the index so school can be used for merging later
avg_scores_df = pd.DataFrame({"avg_math_score": school_group['math_score'].mean(),
                              "avg_reading_score": school_group['reading_score'].mean()}).reset_index()

# create 'super table' of schools_df, students passing reading, students passing math and average math scores.
# we'll pull columns from this table to build the summary.
# normally, merge works on two dataframes, but we'll use a little dot-notation magic to do it all in one statement
super_school_df = pd.merge(schools_df, avg_scores_df, on = 'school') \
.merge(students_passing_math_by_school, on = 'school') \
.merge(students_passing_reading_by_school, on = 'school') 

#calculate percentage of students passing math
pct_students_passing_math_school = (super_school_df['nbr_students_passing_math'] / super_school_df['size']) * 100

#calculate percentage of students passing reading
pct_students_passing_reading_school = (super_school_df['nbr_students_passing_reading'] / \
                                      super_school_df['size']) * 100

#calculate overall passing rate
overall_passing_rate_school = ((pct_students_passing_math_school + pct_students_passing_reading_school) / 2)

# create district_summary dataframe for display
school_summary_df = pd.DataFrame({'School Name': super_school_df['school'],
                               'School Type': super_school_df['type'],
                               'Total Students': super_school_df['size'],
                               'Total School Budget': super_school_df['budget'],
                               'Per Student Budget': (super_school_df['budget'] / super_school_df['size']),
                               'Average Math Score': super_school_df['avg_math_score'],
                               'Average Reading Score': super_school_df['avg_reading_score'],
                               '% Passing Math': pct_students_passing_math_school,
                               '% Passing Reading': pct_students_passing_reading_school,
                               'Overall Passing Rate': overall_passing_rate_school})

school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.2f}'.format)

school_summary_df = school_summary_df[['School Name', 'School Type', 'Total Students', 'Total School Budget',
                                 'Per Student Budget', 'Average Math Score', 'Average Reading Score',
                                 '% Passing Math', '% Passing Reading', 'Overall Passing Rate']] \
                 .set_index('School Name').rename_axis(None)
               
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


# Top Performing Schools (By Passing Rate)

In [29]:
top_performing_schools_df = school_summary_df.nlargest(5, 'Overall Passing Rate')
top_performing_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


# Bottom Performing Schools (By Passing Rate)

In [30]:
bottom_performing_schools_df = school_summary_df.nsmallest(5, 'Overall Passing Rate')
bottom_performing_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


# Math Scores By Grade

In [71]:
#use groupby to create a dataset grouped by high school and grade, then average the columns and reset the indexes
#so we can reuse the school and grade columns later
avg_scores_by_high_school_df = students_df.groupby(['school', 'grade']).mean().reset_index()



#use loc to isolate individual grades
ninth_grade_df = avg_scores_by_high_school_df.loc[avg_scores_by_high_school_df['grade'] == '9th']
tenth_grade_df = avg_scores_by_high_school_df.loc[avg_scores_by_high_school_df['grade'] == '10th']
eleventh_grade_df = avg_scores_by_high_school_df.loc[avg_scores_by_high_school_df['grade'] == '11th']
twelfth_grade_df = avg_scores_by_high_school_df.loc[avg_scores_by_high_school_df['grade'] == '12th']

#reduce dataframes to school, math score, reading score
ninth_grade_red_df = pd.DataFrame({"school": ninth_grade_df["school"],
                                   "reading_score_9th": ninth_grade_df['reading_score'],
                                   "math_score_9th": ninth_grade_df['math_score']})
tenth_grade_red_df = pd.DataFrame({"school": tenth_grade_df["school"],
                                   "reading_score_10th": tenth_grade_df['reading_score'],
                                   "math_score_10th": tenth_grade_df['math_score']})
eleventh_grade_red_df = pd.DataFrame({"school": eleventh_grade_df["school"],
                                   "reading_score_11th": eleventh_grade_df['reading_score'],
                                   "math_score_11th": eleventh_grade_df['math_score']})
twelfth_grade_red_df = pd.DataFrame({"school": twelfth_grade_df["school"],
                                   "reading_score_12th": twelfth_grade_df['reading_score'],
                                   "math_score_12th": twelfth_grade_df['math_score']})

all_scores_by_grade_hs_df = pd.merge(ninth_grade_red_df, tenth_grade_red_df, on = 'school')\
.merge(eleventh_grade_red_df, on = 'school')\
.merge(twelfth_grade_red_df, on = 'school') 
            

math_scores_by_grade_df = pd.DataFrame({'school': all_scores_by_grade_hs_df['school'],
                                        '9th': all_scores_by_grade_hs_df['math_score_9th'],
                                        '10th': all_scores_by_grade_hs_df['math_score_10th'],
                                        '11th': all_scores_by_grade_hs_df['math_score_11th'],
                                        '12th': all_scores_by_grade_hs_df['math_score_12th']})

math_scores_by_grade_df = math_scores_by_grade_df[['school', '9th', '10th', '11th', '12th']]\
                 .set_index('school').rename_axis(None)
math_scores_by_grade_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores By Grade

In [72]:
#This table uses the average score and grade-level datasets from Math Scores By Grade to reduce processing
reading_scores_by_grade_df = pd.DataFrame({'school': all_scores_by_grade_hs_df['school'],
                                        '9th': all_scores_by_grade_hs_df['reading_score_9th'],
                                        '10th': all_scores_by_grade_hs_df['reading_score_10th'],
                                        '11th': all_scores_by_grade_hs_df['reading_score_11th'],
                                        '12th': all_scores_by_grade_hs_df['reading_score_12th']})

reading_scores_by_grade_df = reading_scores_by_grade_df[['school', '9th', '10th', '11th', '12th']]\
                 .set_index('school').rename_axis(None)
reading_scores_by_grade_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160
